In [ ]:
# NUMPY para cálculos numéricos entre arreglos
import numpy as np

# MATPLOTLIB para graficar
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline

In [ ]:
# LLENA LA MATRIZ "MANUALMENTE"
def Laplaciano2D(Nx, Ny, diag):
    """ Esta funcion calcula los coeficientes del 
    sistema lineal producido por el operador de 
    Laplace en 2D. Estos coeficientes son almacenados 
    en la matriz pentadiagonal correspondiente."""
    N = Nx * Ny
    A = np.zeros((N,N))

# Primero llena los bloques tridiagonales
    for j in range(0,Ny):
        ofs = Nx * j
        
        A[ofs, ofs] = diag; 
        A[ofs, ofs + 1] = -1
        for i in range(1,Nx-1):
            A[ofs + i, ofs + i]     = diag
            A[ofs + i, ofs + i + 1] = -1
            A[ofs + i, ofs + i - 1] = -1
        A[ofs + Nx - 1, ofs + Nx - 2] = -1
        A[ofs + Nx - 1, ofs + Nx - 1] = diag 

# Despues llena las dos diagonales externas
    for k in range(0,N-Nx):
        A[k, Nx + k] = -1
        A[Nx + k, k] = -1
        
    return A

In [ ]:
# Datos del problema
N = 20
Nx = N
Ny = N
a = 0
b = 1
h = (b-a)/(N+1)
h2= h * h

In [ ]:
# Malla (fina)
x = np.linspace(0,1,Nx+2)
y = np.linspace(0,1,Ny+2)
xg, yg = np.meshgrid(x,y)
#print(xg)
#print(yg)

In [ ]:
# Solución analítica
zg = np.exp(xg * yg)
#print(zg)

In [ ]:
# Grafica la solución analítica
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$u$')
ax.set_title('Solución analítica $u(x) = e^{xy}$')
ax.set_zlim(-0.01, 3.0)
surf = ax.plot_surface(xg, yg, zg, cmap=cm.coolwarm)
fig.colorbar(surf, shrink=0.75)

In [ ]:
# Arreglo para almacenar la solución numérica en la malla
u = np.ones((Ny+2, Nx+2))
#print(u)

In [ ]:
# Condiciones de frontera
u[ 0, :] = np.exp(x * 0) # y = 0
u[-1, :] = np.exp(x * 1) # y = 1
u[ :, 0] = np.exp(0 * y) # x = 0
u[ :,-1] = np.exp(1 * y) # x = 1

#print(u)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$u$')
ax.set_title('Condiciones de frontera $u_{\partial \Omega}(x) = e^{xy}$')
ax.set_zlim(-0.01, 3.0)
surf = ax.plot_surface(xg, yg, u, cmap=cm.coolwarm)

In [ ]:
# RHS del sistema lineal
xs = xg[1:-1,1:-1]
ys = yg[1:-1,1:-1]

f = (1 - xs*xs - ys*ys) * np.exp(xs * ys) * h2

# Aplicación de las condiciones de frontera
f[ 0, :] += u[ 0, 1:-1] # y = 0
f[-1, :] += u[-1, 1:-1] # y = 1
f[ :, 0] += u[ 1:-1, 0] # x = 0
f[ :,-1] += u[ 1:-1,-1] # x = 1

#print(f)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$f$')
ax.set_title('RHS: $f(x,y) = [(1-x^2-y^2)e^{xy}] * h^2$')
#ax.set_zlim(-3.0, 3.0)
ax.view_init(30, -40)
surf = ax.plot_surface(xs, ys, f, cmap=cm.coolwarm)

In [ ]:
# Matriz del sistema lineal
A = Laplaciano2D(Nx, Ny, 4 + h2) # Matriz del sistema
print(A)

In [ ]:
# Preparación para obtener la solución del sistema lineal
ut = np.copy(u[1:Ny+1,1:Nx+1])
ut.shape = ut.size   # Cambiamos los arreglos a formato unidimensional
f.shape = f.size # Cambiamos los arreglos a formato unidimensional

#print(ut)
#print(f)

In [ ]:
import time
t1_start = time.perf_counter()
ut = np.linalg.solve(A,f)
t1_stop = time.perf_counter()
#print(ut)
print(time.ctime(), '\n CPUT time: {:0.6f} \n '.format(t1_stop-t1_start))

In [ ]:
ut.shape = (Ny, Nx) # Regresamos el arreglo a formato bidimensional
u[1:Ny+1,1:Nx+1] = ut
#print(u)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$u$')
ax.set_title('Solución numérica $u(x)$')
ax.set_zlim(-0.01, 3.0)
surf = ax.plot_surface(xg, yg, u, cmap=cm.coolwarm)
fig.colorbar(surf, shrink=0.75)